# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

In [1]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [2]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [3]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

biology_template = """你是一位很棒的生物学家。你擅长回答生物学相关的问题。
之所以如此出色，是因为你能够将复杂的生物现象分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

computer_template = """你是一位很棒的计算机学家。你擅长回答计算机科学相关的问题。
之所以如此出色，是因为你能够将计算机科学中的难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

philology_template = """你是一位很棒的语文学家。你擅长回答与语言和文学相关的问题。
之所以如此出色，是因为你能够将复杂的语言现象分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""


In [4]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "语文",
        "description": "适用于回答语文问题",
        "prompt_template": philology_template,
    },
]

In [5]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct", base_url="https://api.xiaoai.plus/v1")

In [6]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain
default_chain = ConversationChain(llm=llm, output_key="text")

G:\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
G:\miniconda3\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `ConversationChain` was deprecated in LangChain 0.2.7 and will be removed in 1.0. Use RunnableWithMessageHistory: https://api.python.langchain.com/en/latest/runnables/langchain_core.runnables.history.RunnableWithMessageHistory.html instead.
  warn_deprecated(


In [7]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [8]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [9]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [10]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '语文: 适用于回答语文问题']


In [11]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
语文: 适用于回答语文问题


In [12]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [13]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
语文: 适用于回答语文问题

<< INPUT >>
{input}

<< OUTP

In [14]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [15]:
print(chain.invoke("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '什么是黑体辐射？'}
> Finished chain.
{'input': '什么是黑体辐射？', 'text': '\n\n黑体辐射是一种理想化的物理模型，它假设物体能够吸收和发射所有波长的电磁辐射，并且吸收的辐射能量完全转化为热能。在热力学中，黑体辐射被用来描述实际物体的辐射特性，它的能量密度和频率的关系由普朗克定律和斯特藩-玻尔兹曼定律来描述。黑体辐射也是研究物体热平衡和热辐射的重要工具。'}


In [16]:
print(
    chain.invoke(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': 'What is the first prime number greater than 40, such that adding one to this prime number is divisible by 3?'}
> Finished chain.
{'input': 'What is the first prime number greater than 40, such that adding one to this prime number is divisible by 3?', 'text': '\n\n这个问题可以分解为以下几个部分：\n\n1. 找出大于40的第一个素数。\n2. 在这个素数的基础上加1。\n3. 确认加1后的数能否被3整除。\n\n接下来，我们来解决这些问题：\n\n1. 大于40的素数是43。\n2. 43加1等于44。\n3. 44除以3余数为2，所以44加1不可以被3整除。\n\n因此，大于40的第一个素数，且加1后可以被3整除的数是47。'}


In [17]:
router_chain.verbose = True

In [18]:
print(chain.invoke("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '什么是黑洞？'}
> Finished chain.
{'input': '什么是黑洞？', 'text': '\n\n黑洞是宇宙中最神秘的天体之一，它是由巨大的质量和密度所形成的。一般来说，当一颗恒星燃烧完毕并死亡时，它会变成一种非常致密的物质，称为中子星。但是，如果恒星的质量非常巨大，它会塌缩到一个无法想象的极点，形成一个非常强大的引力场，这就是黑洞。黑洞的引力极其强大，甚至连光也无法逃离它的吸引，因此它表面是一片黑暗，无法被直接观测到。我们目前对黑洞的了解还非常有限，但它们在宇宙中扮演着重要的角色，对于我们理解宇宙的结构'}


### Homework

#### 扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains

In [19]:
# 1 生物
print(chain.invoke("什么是DNA？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '什么是DNA？'}
> Finished chain.
{'input': '什么是DNA？', 'text': '\n\nDNA（脱氧核糖核酸）是生物体内存储遗传信息的分子，它是由四种碱基（腺嘌呤、鸟嘌呤、胞嘧啶和鸟苷）组成的长链状分子。它存在于细胞核和线粒体内，也被称为基因的载体。DNA通过其特殊的序列编码了生物体的遗传信息，决定了生物体的生长、发育和功能。DNA的结构是双螺旋状，由两条互补的链组成，这种结构使得DNA能够自我复制，并且在细胞分裂时能够传递遗传信息。DNA的发现和研究对于生物学的发展和进化有着重大的意义。'}


In [20]:
# 2 计算机
print(chain.invoke("什么是计算机网络技术？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '什么是计算机网络技术？'}
> Finished chain.
{'input': '什么是计算机网络技术？', 'text': '\n\n计算机网络技术是通过使用硬件设备和软件程序，将多台计算机连接起来，实现数据传输和通信的技术。它可以让多台计算机之间能够共享资源，如文件、打印机、数据库等，同时也可以让用户通过网络进行远程控制和交互。计算机网络技术还包括网络协议的设计和实现，网络安全和管理等方面，是现代信息技术发展中不可或缺的重要组成部分。它的发展和应用促进了信息的快速传播和交流，极大地改变了人们的生活和工作方式。'}


In [21]:
# 3 语文
print(chain.invoke("论语对中国文学发展的影响是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
语文: {'input': '论语对中国文学发展的影响是什么？'}
> Finished chain.
{'input': '论语对中国文学发展的影响是什么？', 'text': '\n\n这个问题涉及到语言和文学两个方面，我们可以先从语言的角度来回答。论语作为儒家经典之一，对中国的语言文化产生了深远的影响。首先，它的语言简练明了，易于理解，使得它成为后世文言文的典范。其次，论语中的许多格言和箴言被后人广泛引用，使得它们成为中国文化的重要组成部分。再者，论语中所提倡的“仁义”、“礼让”等思想也影响了中国人的思维方式和语言表达，使得中国语言文化更具有温和、包容的特点。\n\n从文学的角度来看，论语也对中国文学发展产生了重大的影响。首先，论语作为一部儒家经典，为'}
